In [19]:
import os
import requests
from dotenv import load_dotenv
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from datetime import datetime
from langchain.agents import create_tool_calling_agent, AgentExecutor

import pprint as pp

from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [20]:
SECTORS_API_KEY = os.getenv("SECTORS_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [21]:
def get_info(url):
    url = url
    print(url)
    headers = {
        "Authorization": f"{SECTORS_API_KEY}",
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        return {"error": "Failed to get data", "reason": e}


In [22]:
@tool
def get_current_date_ai():
    """
    Get current date
    """
    return datetime.now().strftime("%Y-%m-%d")

In [23]:
@tool
def get_company_information_ai(symbol: str, section: str):
    """
    :param symbol: company symbol with length of 4 uppercase character. format: XXXX
    :param section: Get company information of any section from:
        - Overview: General knowledge about the company, provides information about the company. [industry, subsector, market cap, sector, symbol, addressm employee num, website, phone]
        - Valuation: The extent of the companys wealth/assets, measured with specific criteria. [pe, pb, ps, peg ratio]
        - Peers: Companies that have a similar market share, including their respective market capitalizations. [group name, peers companies]
        - Future: Analysis of the company's current financial state and potential future prospects. [forecast, growth]
        - Financials: The company's revenue and earnings. [tax, revenue, earnings, debt, asset, profits, equity, liabilities]
        - Dividend: Stock payout ratings. [total yield, total divident]
        - Management: The managers/chiefs present at BRI. [executive, shareholder]
        - Ownership: Distribution of the companys ownership. [major shareholder, ownership percentage, monthly net transaction]
    """
    url = f"https://api.sectors.app/v1/company/report/{symbol.lower()}/?sections={section.lower()}"
    return get_info(url)


In [24]:
@tool
def top_company_by_transaction_volume_ai(
    start_date: str,
    end_date: str,
    sub_sector: str = "",
    n: int = 5,
):
    """
    Get top n of most traded company by transaction volume in certain period of time.
    
    use get_current_date_ai to get current date
    :param start_date: Start date of the period of time. format: yyyy-mm-dd.
    :param end_date: End date of the period of time. format: yyyy-mm-dd. date must be greater than start date.
    use get_current_date_ai result to get calculate start_date and end_date
    
    :param sub_sector: Sub sector of the company. all availabe sub sector can be get by calling get_available_subsectors_ai
    :param n: Number of top company. default is 5
    :return: List of top n company by transaction volume
    """
    start_date = datetime.strptime(start_date, "%Y-%m-%d").strftime("%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d").strftime("%Y-%m-%d")
    print(start_date, end_date)
    if sub_sector == "":
        url = f"https://api.sectors.app/v1/most-traded/?start={start_date}&end={end_date}&n_stock={n}"
    else:
        url = f"https://api.sectors.app/v1/most-traded/?start={start_date}&end={end_date}&n_stock={n}&sub_sector={'-'.join(sub_sector.lower().split())}"
    print(url)
    return get_info(url)

In [25]:
@tool
def get_daily_transaction_ai(symbol: str, start_date: str, end_date: str):
    """
    Return daily transaction data of a given symbol on a certain interval
    contains closing prices, volume, and market cap
    
    :param symbol: company symbol, the first 4, symbol of company
    :param start_date: start date of the interval
    :param end_date: end date of the interval
    use get_current_date_ai result to get calculate start_date and end_date
    
    use tool get_info_ai for more information about the company.
    """
    url = (
        f"https://api.sectors.app/v1/daily/{symbol}/?start={start_date}&end={end_date}"
    )
    return get_info(url)

In [26]:
@tool
def get_performance_of_company_since_ipo_listing_ai(symbol: str):
    """
    Get the performance of the company since its IPO listing.
    :param symbol: company symbol, the first 4, symbol of company
    
    use tool get_info_ai for more information about the company.
    """
    url = f"https://api.sectors.app/v1/listing-performance/{symbol}/"
    return get_info(url)

In [27]:
@tool
def get_revenue_and_cost_segment(symbol: str):
    """
    Get revenue and cost segment of a company [revenue breakdown, value, source, target]
    :param symbol: company symbol, the first 4, symbol of company
    
    use tool get_info_ai for more information about the company.
    """
    url = f"https://api.sectors.app/v1/company/financials/{symbol}/"
    return get_info(url)

In [28]:
tools = [top_company_by_transaction_volume_ai, get_company_information_ai, get_current_date_ai, get_daily_transaction_ai, get_performance_of_company_since_ipo_listing_ai, get_revenue_and_cost_segment]

In [29]:

llm = ChatGroq(
    temperature=0.05,
    model_name="llama3-groq-8b-8192-tool-use-preview",
    groq_api_key=GROQ_API_KEY,
)
llm.bind_tools(tools)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x123b35ad0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x123b627d0>, model_name='llama3-groq-8b-8192-tool-use-preview', temperature=0.05, groq_api_key=SecretStr('**********'))

In [30]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"""
            Respond logically and according to the tools provided. Answer it as humanly as possible.
            FOR ALL QUESTION, RUN get_current_date_ai tool FIRST and figure it out about yesterday, tomorrow, or n of days ago with format yyyy-mm-dd
            if ask about company, use get_company_information_ai tool to get the information about the company.
            
            If two different tools were used, use the first tool to get the information and the second tool to get the specific information, and so on.
            procedure on how to use multiple tool:
            question: Give me the no 1 most biggest volume transaction in ist august 2019, and give me the overall information about the company
                - use top_company_by_transaction_volume_ai to get the company and its 4 letter symbol
                - use 4 letter symbol in get_company_information_ai to get the overview
                - return the result
            """,
        ),
        # ("ai", "{chat_history}"),
        ("human", "{input}"),
        # msg containing previous agent tool invocations
        # and corresponding tool outputs
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [31]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")
memory

ConversationBufferMemory(memory_key='chat_history')

In [32]:
agent = create_tool_calling_agent(llm, tools, prompt) 
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n            Respond logically and according to the tools provided. Answer it as humanly as possible.\n            FOR ALL QUESTION, RUN get_current_date_ai tool FIRST and figure it out about yesterday, tomorrow, or n of days ago with format yyyy-mm-dd\n            if ask about company, use get_company_information_ai tool to get the information about the company.\n            \n            If two di

In [33]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)
agent_executor

AgentExecutor(memory=ConversationBufferMemory(memory_key='chat_history'), verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n            Respond logically and according to the tools provided. Answer it as humanly as possible.\n            FOR ALL QUESTION, RUN get_current_date_ai tool FIRST and figure it out about yesterday, tomorrow, or n of days ago with format yyyy-mm-dd\n            if ask a

In [34]:
query_0 = "Give me the top 1 company by transaction volume in 2 august 2023. give me the email of said company"
query_1 = "What are the top 5 company by transaction volume on the first of this month?"
query_2 = "What are the most traded stock yesterday?"
query_3 = (
    "What are the top 7 most traded stocks between 6th June to 10th June this year?"
)
query_4 = "What are the top 3 companies by transaction volume over the last 7 days?"  # gak konsisten atara pakai angka dan tidak
query_5 = "Based on the closing prices of BBCA between 1st and 30th of June 2024, are we seeing an uptrend or downtrend? Try to explain why."
query_6 = "What is the company with the largest market cap between BBCA and BREN? For said company, retrieve the email, phone number, listing date and website for further research."
query_7 = "What is the performance of GOTO (symbol: GOTO) since its IPO listing?"
query_8 = "If i had invested into GOTO vs BREN on their respective IPO listing date, which one would have given me a better return over a 90 day horizon?"

In [35]:
queries = [query_4]

In [36]:
for query in queries:
    print("Question:", query)
    result = agent_executor.invoke({"input": query})
    print("Answer:", "\n", result["output"], "\n\n======\n\n")

Question: What are the top 3 companies by transaction volume over the last 7 days?


> Entering new AgentExecutor chain...

Invoking: `get_current_date_ai` with `{}`


2024-08-29
Invoking: `top_company_by_transaction_volume_ai` with `{'start_date': '2023-09-01', 'end_date': '2023-09-07', 'n': 3}`


2023-09-01 2023-09-07
https://api.sectors.app/v1/most-traded/?start=2023-09-01&end=2023-09-07&n_stock=3
https://api.sectors.app/v1/most-traded/?start=2023-09-01&end=2023-09-07&n_stock=3
{'2023-09-01': [{'symbol': 'GOTO.JK', 'company_name': 'PT GoTo Gojek Tokopedia Tbk', 'volume': 1217262200, 'price': 95}, {'symbol': 'MEDC.JK', 'company_name': 'PT Medco Energi Internasional Tbk', 'volume': 586829900, 'price': 1290}, {'symbol': 'MUTU.JK', 'company_name': 'PT Mutuagung Lestari Tbk', 'volume': 547390500, 'price': 189}], '2023-09-04': [{'symbol': 'BUMI.JK', 'company_name': 'Bumi Resources Tbk', 'volume': 1139423700, 'price': 145}, {'symbol': 'DEWA.JK', 'company_name': 'Darma Henwa Tbk', 'volume':